# Initial test with spectral signal of gene trees

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import sys, subprocess, time, dendropy, os
import numpy as np

bindir="/home/leo/local/bin/"
localdir="/tmp/"

In [147]:
def run_dspr (tree1, tree2):
    localfile = localdir + "pair.tre"
    dendropy.TreeList([tree1,tree2]).write(path=localfile, schema="nexus")
    start_time = time.perf_counter()
    proc_run = subprocess.run(bindir + "gf_spr_distance " + localfile, shell=True, stdout=subprocess.PIPE, 
                              universal_newlines=True)
    elapsed_time = time.perf_counter() - start_time
    line = proc_run.stdout.splitlines()[1].split()[1:4] # second line has dists as columns, we want 2nd~4th
    dists = [float(i) for i in line] # SPR, RF, Hdist, respectively
        
    return elapsed_time, dists

In [7]:
n_leaves = 8
def generate_trees (n_trees, n_spr, frac_wrong, max_error):
    os.chdir(localdir)
    proc_run = subprocess.run([bindir + "gf_generate_spr_trees", str(n_leaves), str(n_trees), str(n_spr)],
                              stdout=subprocess.PIPE, universal_newlines=True)
    with open("tmp.tre", "w") as trefile:
        print(proc_run.stdout, file=trefile)
    proc_run = subprocess.run([bindir + "bmc2_addTreeNoise", "tmp.tre", str(frac_wrong), str(max_error), "100"],
                              stdout=subprocess.PIPE, universal_newlines=True)
    return dendropy.TreeList.get(path="noise.tre", schema="nexus")

In [8]:
x = generate_trees (2,1,0.5,0.8)

NewickReaderIncompleteTreeStatementError: Error parsing data source 'noise.tre' on line 14 at column 65: Incomplete or improperly-terminated tree statement (last character read was ')' instead of a semi-colon ';')